## **安装conda，后面需要用conda来安装各种必须的library**

In [2]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'chmod' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'bash' 不是内部或外部命令，也不是可运行的程序
或批处理文件。



Examining rdkit:  80%|████████  | 4/5 [00:00<00:00, 36.78it/s]                         
                                                              

Examining conflict for python rdkit:   0%|          | 0/5 [00:00<?, ?it/s]
                                                                          

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - rdkit -> python[version='2.7.*|3.5.*|3.6.*|>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|3.4.*']

Your python: python=3.8

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that conda will not
change your python version to a different minor version unless you explicitly specify
that.






Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed


**安装本次训练的关键deepchem, 你也可直接从github/deepchem安装**

In [3]:
!pip install --pre deepchem

**安装可视化分析工具；注意在谷歌colab即使按照以下工具也无法显示，显示功能只有在非colab环境下的Jupyter notebook 才有作用**

In [3]:
!pip install -q mdtraj nglview

     |████████████████████████████████| 6.2 MB 12.1 MB/s 
     |████████████████████████████████| 5.7 MB 19.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 121 kB 80.6 MB/s 
     |████████████████████████████████| 243 kB 66.7 MB/s 
     |████████████████████████████████| 787 kB 96.7 MB/s 
     |████████████████████████████████| 124 kB 85.9 MB/s 
     |████████████████████████████████| 178 kB 88.2 MB/s 
     |████████████████████████████████| 2.2 MB 89.8 MB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
     |████████████████████████████████| 1.0 MB 74.2 MB/s 
     |████████████████████████████████| 1.4 MB 73.3 MB/s 
     |████████████████████████████████| 370 kB 87.6 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 428 kB 75.0 MB/s 
     |████████████████████████████████| 1.9 MB 70.4 MB/s 
     |█

**安装开源的动力学模拟软件openmm,为下一步安装pdbfixer提供环境**

In [4]:
!yes | conda install -c conda-forge openmm 

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - openmm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            1_gnu          22 KB  conda-forge
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    conda-4.10.3               |   py37h89c1867_1         3.0 MB  conda-forge
    cudatoolkit-10.2.89        |       hfd86e86_1       365.1 MB
    fftw-3.3.10               

**在openmm 框架的基础上，安装蛋白结构修复和优化所需的pdbfixer**

In [5]:
!yes | conda install -c conda-forge pdbfixer

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pdbfixer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pdbfixer-1.8.1             |     pyh6c4a22f_0         498 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         498 KB

The following NEW packages will be INSTALLED:

  pdbfixer           conda-forge/noarch::pdbfixer-1.8.1-pyh6c4a22f_0


Proceed ([y]/n)? 

pdbfixer-1.8.1       | 498 KB    | : 100% 1.0/1 [00:00<00:00,  6.16it/s]
Preparing transaction: \ done
Verifying transaction: / done
Executing transaction: \ | / - done


**导入所有所需的环境和工具**


In [6]:
import os
import numpy as np
import pandas as pd

import tempfile

from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc

from deepchem.utils import download_url, load_from_disk

**下载包含了众多小分子smiles的csv文件，下载pdb共晶文件。文件均被下载到内置路径，左侧的文件侧边栏看不到（如果使用colab的话）**

In [7]:
data_dir = dc.utils.get_data_dir()
dataset_file = os.path.join(data_dir, "pdbbind_core_df.csv.gz")

if not os.path.exists(dataset_file):
    print('File does not exist. Downloading file...')
    download_url("https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/pdbbind_core_df.csv.gz")
    print('File downloaded...')

raw_dataset = load_from_disk(dataset_file)
raw_dataset = raw_dataset[['pdb_id', 'smiles', 'label']]

File does not exist. Downloading file...
File downloaded...


**显示前两行**

In [8]:
from simtk.openmm.app import PDBFile
from pdbfixer import PDBFixer

from deepchem.utils.vina_utils import prepare_inputs

In [9]:
# consider one protein-ligand complex for visualization
pdbid = raw_dataset['pdb_id'].iloc[1]
ligand = raw_dataset['smiles'].iloc[1]

**接下来使用pdbfixer修复所有的蛋白共晶结构;使用RDKIT进一步清洗结构**

In [10]:
%%time
fixer = PDBFixer(pdbid=pdbid)
PDBFile.writeFile(fixer.topology, fixer.positions, open('%s.pdb' % (pdbid), 'w'))

p, m = None, None
# fix protein, optimize ligand geometry, and sanitize molecules
try:
    p, m = prepare_inputs('%s.pdb' % (pdbid), ligand)
except:
    print('%s failed PDB fixing' % (pdbid)) 

if p and m:  # protein and molecule are readable by RDKit
    print(pdbid, p.GetNumAtoms())
    Chem.rdmolfiles.MolToPDBFile(p, '%s.pdb' % (pdbid))
    Chem.rdmolfiles.MolToPDBFile(m, 'ligand_%s.pdb' % (pdbid))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  import sys


3cyx 1415
CPU times: user 3.87 s, sys: 50.8 ms, total: 3.92 s
Wall time: 4.12 s


**图形显示（仅在colab 之外可以发挥作用）**

In [11]:
import mdtraj as md
import nglview

from IPython.display import display, Image

In [12]:
protein_mdtraj = md.load_pdb('3cyx.pdb')
ligand_mdtraj = md.load_pdb('ligand_3cyx.pdb')

In [13]:
v = nglview.show_mdtraj(ligand_mdtraj)

In [14]:
display(v)

NGLWidget()

In [15]:
view = nglview.show_mdtraj(protein_mdtraj)
display(view)

NGLWidget()

**分子对接，所需时间稍长，colab 需要约20分钟**

**DeepChem通过自己的算法自动寻找结合口袋，该算法是基于convex hull method **


In [16]:
finder = dc.dock.binding_pocket.ConvexHullPocketFinder()
pockets = finder.find_pockets('3cyx.pdb')
len(pockets)  # number of identified pockets

37

**对接构象生成是分子对接的关键一步，这里，deepChem利用的是AutoDock Vina来完成这个任务**

In [17]:
vpg = dc.dock.pose_generation.VinaPoseGenerator()

In [18]:
!mkdir -p vina_test

In [19]:
%%time
complexes, scores = vpg.generate_poses(molecular_complex=('3cyx.pdb', 'ligand_3cyx.pdb'),  # protein-ligand files for docking,
                                       out_dir='vina_test',
                                       generate_scores=True
                                      )

Mol [H]OC1=C([H])C([H])=C(C([H])([H])[C@@]2([H])C(=O)N([H])[C@@]([H])(C([H])([H])C(=O)O)C(=O)N([H])[C@@]([H])(C([H])([H])C([H])([H])C(=O)N([H])[H])C(=O)N([H])[C@@]([H])([C@@]([H])(C([H])([H])[H])C([H])([H])C([H])([H])[H])C(=O)N([H])[C@@]([H])([C@@]([H])(C([H])([H])[H])C([H])([H])C([H])([H])[H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N3C([H])([H])C([H])([H])C([H])([H])[C@@]3([H])C(=O)N([H])[C@]([H])(C(=O)N3C([H])([H])C([H])([H])C([H])([H])[C@@]3([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])

CPU times: user 9.19 s, sys: 1.05 s, total: 10.2 s
Wall time: 24min 28s


In [21]:
scores

[-9.2, -9.0, -8.7, -8.6, -8.6, -8.6, -8.4, -8.4, -8.4]

**下面的可可视话仅在colab之外可以**

In [22]:
complex_mol = Chem.CombineMols(complexes[0][0], complexes[0][1])

In [23]:
v = nglview.show_rdkit(complex_mol)
display(v)

NGLWidget()

**分子对接方式二，更简约的方式，这在本质上与上面的过程相同，我们不再运行**

In [ ]:
#docker = dc.dock.docking.Docker(pose_generator=vpg)
#posed_complex, score = next(docker.dock(molecular_complex=('3cyx.pdb', 'ligand_3cyx.pdb'),
                                          use_pose_generator_scores=True))

Mol [H]OC1=C([H])C([H])=C(C([H])([H])[C@@]2([H])C(=O)N([H])[C@@]([H])(C([H])([H])C(=O)O)C(=O)N([H])[C@@]([H])(C([H])([H])C([H])([H])C(=O)N([H])[H])C(=O)N([H])[C@@]([H])([C@@]([H])(C([H])([H])[H])C([H])([H])C([H])([H])[H])C(=O)N([H])[C@@]([H])([C@@]([H])(C([H])([H])[H])C([H])([H])C([H])([H])[H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N3C([H])([H])C([H])([H])C([H])([H])[C@@]3([H])C(=O)N([H])[C@]([H])(C(=O)N3C([H])([H])C([H])([H])C([H])([H])[C@@]3([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])

In [20]:
raw_dataset.head(2)

,pdb_id,smiles,label
0,2d3u,CC1CCCCC1S(O)(O)NC1CC(C2CCC(CN)CC2)SC1C(O)O,6.92
1,3cyx,CC(C)(C)NC(O)C1CC2CCCCC2C[NH+]1CC(O)C(CC1CCCCC...,8.00


In [24]:
pdbids = raw_dataset['pdb_id'].values
ligand_smiles = raw_dataset['smiles'].values

**接下来，将使用约100个蛋白-小分子共晶复合物来训练我们的模型。如果想有更好的训练结果，请考虑使用更大的数据集（4852个共晶），或者考虑使用PDBbind中的所有结构。下面首先从rcsb 网站下载了蛋白共晶，然后用之前安装的pdbfixer来准备蛋白结构，包括替换非天然氨基酸，去除水分子，去除无关小分子等。蛋白共晶被拆分成单独的蛋白质，和单独的小分子**

In [25]:
%%time
for (pdbid, ligand) in zip(pdbids, ligand_smiles):
  fixer = PDBFixer(url='https://files.rcsb.org/download/%s.pdb' % (pdbid))
  PDBFile.writeFile(fixer.topology, fixer.positions, open('%s.pdb' % (pdbid), 'w'))
  
  p, m = None, None
  # skip pdb fixing for speed
  try:
    p, m = prepare_inputs('%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
                          remove_heterogens=False, remove_water=False,
                          add_hydrogens=False)
  except:
    print('%s failed sanitization' % (pdbid)) 

  if p and m:  # protein and molecule are readable by RDKit
    Chem.rdmolfiles.MolToPDBFile(p, '%s.pdb' % (pdbid))
    Chem.rdmolfiles.MolToPDBFile(m, 'ligand_%s.pdb' % (pdbid))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while

1hfs failed sanitization


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  # Remove the CWD from sys.path while

CPU times: user 3min 20s, sys: 2.06 s, total: 3min 22s
Wall time: 5min 18s


In [27]:
proteins = [f for f in os.listdir('.') if len(f) == 8 and f.endswith('.pdb')]
ligands = [f for f in os.listdir('.') if f.startswith('ligand') and f.endswith('.pdb')]

**对比小分子和蛋白靶点的编码ID，去掉那些没有小分子的蛋白结构**

In [28]:
# Handle failed sanitizations
failures = set([f[:-4] for f in proteins]) - set([f[7:-4] for f in ligands])
for pdbid in failures:
  proteins.remove(pdbid + '.pdb')

In [29]:
len(proteins), len(ligands)

(190, 190)

In [30]:
pdbids = [f[:-4] for f in proteins]
small_dataset = raw_dataset[raw_dataset['pdb_id'].isin(pdbids)]
labels = small_dataset.label

**下面采用的分子指纹是CircularFingerPrint**

In [31]:
fp_featurizer = dc.feat.CircularFingerprint(size=2048)

In [32]:
features = fp_featurizer.featurize([Chem.MolFromPDBFile(l) for l in ligands])

In [33]:
dataset = dc.data.NumpyDataset(X=features, y=labels, ids=pdbids)
train_dataset, test_dataset = dc.splits.RandomSplitter().train_test_split(dataset, seed=42)

**可选择性使用下面的代码，该代码块是使用了4852个共晶结构来进行训练，dc.molnet.load_pdbbind可自动从PDBbind数据库下载数据集。整个过程需要较长时间。**

In [ ]:
# # Uncomment to featurize all of PDBBind's "refined" set
# pdbbind_tasks, (train_dataset, valid_dataset, test_dataset), transformers = dc.molnet.load_pdbbind(
#     featurizer=fp_featurizer, set_name="refined", reload=True,
#     data_dir='pdbbind_data', save_dir='pdbbind_data')

**下面进行学习模型构建，采用随机森林**

In [34]:
from sklearn.ensemble import RandomForestRegressor
from deepchem.utils.evaluate import Evaluator
import pandas as pd

In [35]:
seed = 42 # Set a random seed to get stable results
sklearn_model = RandomForestRegressor(n_estimators=100, max_features='sqrt')
sklearn_model.random_state = seed
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

In [36]:
# use Pearson correlation so metrics are > 0
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

evaluator = Evaluator(model, train_dataset, [])
train_r2score = evaluator.compute_model_performance([metric])
print("RF Train set R^2 %f" % (train_r2score["pearson_r2_score"]))

evaluator = Evaluator(model, test_dataset, [])
test_r2score = evaluator.compute_model_performance([metric])
print("RF Test set R^2 %f" % (test_r2score["pearson_r2_score"]))

RF Train set R^2 0.853606
RF Test set R^2 0.004312


In [37]:
# Compare predicted and true values
list(zip(model.predict(train_dataset), train_dataset.y))[:5]

[(7.083299999999992, 7.4),
 (6.525666666666673, 6.85),
 (4.529499999999994, 3.4),
 (6.616833333333339, 6.72),
 (7.4313999999999885, 11.06)]

In [38]:
list(zip(model.predict(test_dataset), test_dataset.y))[:5]

[(5.924891666666667, 4.21),
 (5.898725000000001, 8.7),
 (6.051199999999999, 6.39),
 (6.4456000000000016, 4.94),
 (6.0634883333333285, 9.21)]

**接下来，我们使用ContancCircularFingerprint 对蛋白-小分子的相互作用模式进行数据化**

In [39]:
fp_featurizer = dc.feat.ContactCircularFingerprint(size=2048)

In [ ]:
features = fp_featurizer.featurize(zip(ligands, proteins))
dataset = dc.data.NumpyDataset(X=features, y=labels, ids=pdbids)
train_dataset, test_dataset = dc.splits.RandomSplitter().train_test_split(dataset, seed=42)

Mol CC[C@H](C)[C@@H]1NC(=O)CNC(=O)[C@H](CC2=CC=C(O)C=C2)NC(=O)CNC(=O)[C@H](CCCCN)NC(=O)[C@H](CO)NC(=O)[C@H](CC(=O)O)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)[C@H](C(C)C)NC(=O)[C@H](CCCCN)NC(=O)[C@H](CCSC)NC(=O)[C@H]([C@@H](C)O)NC(=O)[C@H](CC(=O)O)NC(=O)[C@@H](NC(=O)[C@@H]2CCCN2C(=O)[C@H](CCCCN)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCC(N)=O)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@@H](NC(=O)[C@H](CC2=CC=C(O)C=C2)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](CCSC)NC(=O)[C@@H](NC(=O)[C@H](CO)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC2=CC=C(O)C=C2)NC(=O)[C@H](C)NC(=O)[C@H](CC2=CC=C(O)C=C2)NC(=O)[C@H](CCCCN)NC(=O)CNC(=O)[C@H](CCCCN)NC(=O)[C@H](CO)NC(=O)[C@H](CCCCN)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](C)NC(=O)[C@@H](NC(=O)CNC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](C)NC(=O)[C@@H](NC(=O)[C@@H](NC(=O)[C@H](CCCN2N=C2N)NC(=O)[C@@H](NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)[C@@H](NC(=O)[C@H](CO)NC(=O)[C@@H]2CCCN2C(=O)[C@H](CCC(=O)O)NC(=O)[C@

**接下来又是一个随机森林模型的构建**

In [ ]:
seed = 42 # Set a random seed to get stable results
sklearn_model = RandomForestRegressor(n_estimators=100, max_features='sqrt')
sklearn_model.random_state = seed
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)seed = 42 # Set a random seed to get stable results
sklearn_model = RandomForestRegressor(n_estimators=100, max_features='sqrt')
sklearn_model.random_state = seed
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

evaluator = Evaluator(model, train_dataset, [])
train_r2score = evaluator.compute_model_performance([metric])
print("RF Train set R^2 %f" % (train_r2score["pearson_r2_score"]))

evaluator = Evaluator(model, test_dataset, [])
test_r2score = evaluator.compute_model_performance([metric])
print("RF Test set R^2 %f" % (test_r2score["pearson_r2_score"]))

就本次学习而言，机器学习模型没有预测能力。但本流程展示了，一个基本的操作步骤。